## 2. Search Engine

Now, we want to create two different Search Engines that, given as input a query, return the courses that match the query.

### 2.0 Preprocessing 

### 2.0.0)  Preprocessing the text

First, you must pre-process all the information collected for each MSc by:

1. Removing stopwords
2. Removing punctuation
3. Stemming
4. Anything else you think it's needed
   
For this purpose, you can use the [`nltk library](https://www.nltk.org/).

### 2.0.1) Preprocessing the fees column

Moreover, we want the field ```fees``` to collect numeric information. As you will see, you scraped textual information for this attribute in the dataset: sketch whatever method you need (using regex, for example, to find currency symbol) to collect information and, in case of multiple information, retrieve only the highest fees. Finally, once you have collected numerical information, you likely will have different currencies: this can be chaotic, so let chatGPT guide you in the choice and deployment of an API to convert this column to a common currency of your choice (it can be USD, EUR or whatever you want). Ultimately, you will have a ```float``` column renamed ```fees (CHOSEN COMMON CURRENCY)```.

### 2.1. Conjunctive query
For the first version of the search engine, we narrowed our interest to the __description__ of each course. It means that you will evaluate queries only concerning the course's description.

### 2.1.1) Create your index!

Before building the index, 
* Create a file named `vocabulary`, in the format you prefer, that maps each word to an integer (`term_id`).

Then, the first brick of your homework is to create the Inverted Index. It will be a dictionary in this format:

```
{
term_id_1:[document_1, document_2, document_4],
term_id_2:[document_1, document_3, document_5, document_6],
...}
```
where _document\_i_ is the *id* of a document that contains that specific word.

__Hint:__ Since you do not want to compute the inverted index every time you use the Search Engine, it is worth thinking about storing it in a separate file and loading it in memory when needed.

#### 2.1.2) Execute the query
Given a query input by the user, for example:

```
advanced knowledge
```

The Search Engine is supposed to return a list of documents.

##### What documents do we want?
Since we are dealing with conjunctive queries (AND), each returned document should contain all the words in the query.
The final output of the query must return, if present, the following information for each of the selected documents:

* `courseName`
* `universityName`
* `description`
* `URL`

__Example Output__ for ```advanced knowledge```: (please note that our examples are made on a small batch of the full dataset)

<p align="center">
<img src="img/output1.png" width = 1000>
</p>

If everything works well in this step, you can go to the next point and make your Search Engine more complex and better at answering queries.


### 2.2) Conjunctive query & Ranking score

For the second search engine, given a query, we want to get the *top-k* (the choice of *k* it's up to you!) documents related to the query. In particular:

* Find all the documents that contain all the words in the query.
* Sort them by their similarity with the query.
* Return in output *k* documents, or all the documents with non-zero similarity with the query when the results are less than _k_. You __must__ use a heap data structure (you can use Python libraries) for maintaining the *top-k* documents.

To solve this task, you must use the *tfIdf* score and the _Cosine similarity_. The field to consider is still the `description`. Let's see how.


#### 2.2.1) Inverted index
Your second Inverted Index must be of this format:

```
{
term_id_1:[(document1, tfIdf_{term,document1}), (document2, tfIdf_{term,document2}), (document4, tfIdf_{term,document4}), ...],
term_id_2:[(document1, tfIdf_{term,document1}), (document3, tfIdf_{term,document3}), (document5, tfIdf_{term,document5}), (document6, tfIdf_{term,document6}), ...],
...}
```

Practically, for each word, you want the list of documents in which it is contained and the relative *tfIdf* score.

__Tip__: *TfIdf* values are invariant for the query. Due to this reason, you can precalculate and store them accordingly.

#### 2.2.2) Execute the query

In this new setting, given a query, you get the proper documents (i.e., those containing all the query's words) and sort them according to their similarity to the query. For this purpose, as the scoring function, we will use the Cosine Similarity concerning the *tfIdf* representations of the documents.

Given a query input by the user, for example:
```
advanced knowledge
```
The search engine is supposed to return a list of documents, __ranked__ by their Cosine Similarity to the query entered in the input.

More precisely, the output must contain:
* `courseName`
* `universityName`
* `description`
* `URL`
* The similarity score of the documents with respect to the query (float value between 0 and 1)
  
__Example Output__ for ```advanced knowledge```:

<p align="center">
<img src="img/output2.png" width = 1000>
</p>

In [1]:
import pandas as pd

# Read the TSV data
df = pd.read_csv(
    "TSV/course_1.tsv", sep="\t", index_col=False
)

for i in range(2, 6001):
    try:
        df1 = pd.read_csv(
            "TSV/course_" + str(i) + ".tsv",
            sep="\t",
            index_col=False,
        )
        df1.index += i - 1
        df = pd.concat([df, df1])
    except Exception as e:
        print(i)
        print("Error: ", e)


In [4]:
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

# Removing stopwords
nltk.download('stopwords')

def stopless(text):
    if isinstance(text, str):
        words = word_tokenize(text)
        stop_words = set(stopwords.words('english'))
        filtered_words = [word for word in words if word.lower() not in stop_words]
        return " ".join(filtered_words)
    else:
        return text

df = df.applymap(stopless)



[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/petraudovicic/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [5]:
import string
#removing punctuation
nltk.download('punkt')
def punct(text):
    if isinstance(text, str):
        words = word_tokenize(text)
        filtered_words = [word for word in words if word.lower() not in string.punctuation]
        return " ".join(filtered_words)
    else:
        return text

df = df.applymap(punct)

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/petraudovicic/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [6]:

#stemming
from nltk.stem import PorterStemmer
nltk.download('punkt')
ps = PorterStemmer()
def stem(text):
    if isinstance(text, str):
        words = word_tokenize(text)
        stemmed_words = [ps.stem(word) for word in words]
        return " ".join(stemmed_words)
    else:
        return text
df = df.applymap(stem)

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/petraudovicic/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [7]:
import re

def convert_to_numeric(value):
    value = re.sub(r'eur|sek|chf|gbp|rmb|jpy|qr|[£€]', '', value)
    # Remove commas from the numeric part (e.g., for values like '1,000')
    value = value.replace(',', '')
    value=value.replace(' ', '')
    return float(value)
def find_fees(text):
    if isinstance(text,str):
        pattern = r'\b\d{4}/\d{4}\b'
        text = re.sub(pattern, '', text)
        pattern = r'\b\d{4}/\d{2}\b'
        text = re.sub(pattern, '', text)
        # Define the regular expression pattern for currency symbols and values
        pattern = r'((eur|sek|chf|gbp|rmb|jpy|qr|[£€])\s?\d+(?:[.,\s]\d{3})*(?:[.,]\d{2})?|\d+(?:[.,\s]\d{3})*(?:[.,]\d{2})?\s?(eur|sek|chf|gbp|rmb|jpy|qr|[£€]))'
        # Find all matches in the text
        matches = re.findall(pattern, text)
        # Convert values to numeric format and filter out non-euro values
        numeric_values = []
        exchange_rates = {'SEK': 0.08588, 'GBP': 1.1443, 'CHF': 1.03708, 'JPY': 0.00618, 'QR': 0.25672, "RMB": 0.12892}
        for value in matches:
            value_numeric = convert_to_numeric(value[0])
            if (('€' or 'eur') in value):
                numeric_values.append(value_numeric)
            elif 'sek' in value and exchange_rates is not None and 'SEK' in exchange_rates:
                numeric_values.append(value_numeric * exchange_rates['SEK'])
            elif ('£' or 'gbp') in value and exchange_rates is not None and 'GBP' in exchange_rates:
                numeric_values.append(value_numeric * exchange_rates['GBP'])
            elif 'chf' in value and exchange_rates is not None and 'CHF' in exchange_rates:
                numeric_values.append(value_numeric * exchange_rates['CHF'])
            elif 'jpy' in value and exchange_rates is not None and 'JPY' in exchange_rates:
                numeric_values.append(value_numeric * exchange_rates['JPY'])
            elif 'qr' in value and exchange_rates is not None and 'QR' in exchange_rates:
                numeric_values.append(value_numeric * exchange_rates['QR'])
            else:
                if ('qr' in value and exchange_rates is not None and 'QR' in exchange_rates):
                    numeric_values.append(convert_to_numeric(value) * exchange_rates['RMB'])

        if not numeric_values:
            return None  # No euro values found
    else:
        return text

    # Find the largest value
    max_value = max(numeric_values)

    return max_value
df['converted_fees'] = df['fees'].apply(find_fees)
filtered_df = df.dropna(subset=['converted_fees'])[['fees','converted_fees']]
filtered_df




,fees,converted_fees
0,uk student full time £9450 2022/2023 academ ye...,18880.950
29,tuition fee uk student start full-tim studi 20...,11797.733
49,"uk student integr pre-mast cours £10,500 msc c...",12015.150
50,full time home award fee £11500 home modul fee...,13159.450
51,"€11,000",11000.000
...,...,...
5988,"full time £18,000 part time £9,000",20597.400
5989,"full time £18,000 part time £9,000",20597.400
5993,"full time £14,100 part time £7,050",16134.630
5994,"full time £18,000 part time £9,000",20597.400


In [2]:
vocabulary=set()
df.description.apply(lambda row: [vocabulary.add(word) for word in row])
vocabulary
index = {}
unique_id = 1
for word in list(vocabulary):
  index[unique_id] = word
  unique_id+=1
index

NameError: name 'dataset' is not defined

In [52]:
import os
import csv

def inverted_index(text):
    d=dict({})
    for i in text.split():
         d[i]=[]
    for i in text.split(): #ovo se vjv ne moze ovako
        if ((i in tsv_file[4]) and (tsv_file[0] not in d.get(i, []))): 
                d[i].extend(tsv_file[0])   
    return d

directory_path = '/Users/petraudovicic/Desktop/adm/ADM_HW3/TSV'
# Iterate over all files in the directory
for filename in os.listdir(directory_path):
    if filename.endswith(".tsv"):
        file_path = os.path.join(directory_path, filename)

        # Open the TSV file and read its contents using the csv module
        with open(file_path, 'r', newline='', encoding='utf-8') as tsv_file:
            tsv_reader = csv.reader(tsv_file, delimiter='\t')
            print(inverted_index('advanced knowledge'))

TypeError: '_io.TextIOWrapper' object is not subscriptable